In [1]:
import nba_api
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, leaguegamefinder, playerdashboardbyclutch, playergamelogs, commonplayerinfo, teamplayeronoffdetails
import pandas as pd

In [2]:
class Player(object):
    def __init__(self, PlayerID, TeamID):
        print(6666666666666666666666666666666666666666666666666666666666)
        self.PlayerID =  PlayerID 
        print(7777777777777777777777777777777777777777)
        self.PlayerName = players.find_player_by_id(player_id=self.PlayerID)['full_name']
        print(1111111111111111111111111111111111111111)
        print(self.PlayerName)
        self.TeamID = TeamID
        print(5555555555555555555555555555555555555555)

        self.StartYear = commonplayerinfo.CommonPlayerInfo(player_id= self.PlayerID).get_data_frames()[0]['FROM_YEAR'][0] 
        print("gggggggggggggggggggggg", self.StartYear)
        if self.StartYear < 2010:
            print(88888888888888888888888)
            self.StartYear = 2010
            
        print(99999999999999999999999999999999999999999999999999)
        self.LastYear = 2021
        print(10101010101010101010101010100101010101010101)
        #self.Data = self.compile_active_player_data()

    def compile_active_player_data(self):
        print(222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222)
        player_name = []
        game_date_raw_data = []
        win_loss_raw_data = []
        free_throws_raw_data = [] 
        feild_goals_2pts_raw_data = []
        feild_goals_3pts_raw_data = []
        matchup_raw_data = []
        
        print(self.StartYear, self.LastYear)
        for year in range((self.StartYear), (self.LastYear)):
            print("-------------------------------------------------")
            season = str(year) + "-" + str(((year+1)-2000))
            print("***************************************")
            playerLogs = playergamelogs.PlayerGameLogs(player_id_nullable=self.PlayerID, season_nullable=season)
            print(5656656656566556565656556565656565556565656565656656556565655656566556)
            print(season)
            
            game_date_api_data = playerLogs.get_data_frames()[0]["GAME_DATE"]
            win_loss_api_data = playerLogs.get_data_frames()[0]["WL"]
            free_throws_api_data = playerLogs.get_data_frames()[0]["FTM"]
            feild_goals_2pts_api_data = playerLogs.get_data_frames()[0]["FGM"]
            feild_goals_3pts_api_data = playerLogs.get_data_frames()[0]["FG3M"]
            matchup_api_data = playerLogs.get_data_frames()[0]["MATCHUP"]
            print(767677676767676767676767676776)
            
            
            for game in range(0, len(game_date_api_data)):
                    player_name.append(self.PlayerName)
                    game_date_raw_data.append(game_date_api_data[game])
                    win_loss_raw_data.append(win_loss_api_data[game])
                    free_throws_raw_data.append(free_throws_api_data[game]) 
                    feild_goals_2pts_raw_data.append(feild_goals_2pts_api_data[game])
                    feild_goals_3pts_raw_data.append(feild_goals_3pts_api_data[game])
                    matchup_raw_data.append(matchup_api_data[game])
        print(333333333333333333333333333333333333333333333333333)
        player_data = {
        "Player Name": player_name,
        "Game Date": game_date_raw_data,
        "Win/Loss": win_loss_raw_data,
        "Free Throws": free_throws_raw_data,
        "Feild Goals 2pts": feild_goals_2pts_raw_data,
        "Feild Goals 3pts": feild_goals_3pts_raw_data,
        "Matchup": matchup_raw_data
        }
        print(44444444444444444444444444444444444444444444444444)
        player_dataFrame = pd.DataFrame(data=player_data)
        print(4444)
        print(player_dataFrame)
        return(player_dataFrame)


    def get_active_player_data(self):
        print(1111)
        print("Why is this not printing")
        return(self.compile_active_player_data())

        #def get_active_player_data(self):


    



In [3]:
test = Player(203081).get_active_player_data()
test.get_active_player_data()

6666666666666666666666666666666666666666666666666666666666
7777777777777777777777777777777777777777
1111111111111111111111111111111111111111
Damian Lillard
5555555555555555555555555555555555555555
gggggggggggggggggggggg 2012
99999999999999999999999999999999999999999999999999
10101010101010101010101010100101010101010101
1111
Why is this not printing
222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
2012 2021
-------------------------------------------------
***************************************
5656656656566556565656556565656565556565656565656656556565655656566556
2012-13
767677676767676767676767676776
-------------------------------------------------
***************************************
5656656656566556565656556565656565556565656565656656556565655656566556
2013-14
767677676767676767676767676776
-------------------------------------------------
***************************************
5656656656566556565656556565656565556565656565656

AttributeError: 'DataFrame' object has no attribute 'get_active_player_data'

In [ ]:
class Team(object): 
    def __init__(self,TeamName):
        self.TeamName = TeamName
        self.TeamID = teams.find_teams_by_full_name("Portland Trail Blazers")[0]['id']
        #self.ActivePlayerLists = compile_active_player_DataFrame()

    def compile_team_roster_DataFrame(self):
        print("It")
        active_team_player_raw =teamplayeronoffdetails.TeamPlayerOnOffDetails(team_id=self.TeamID).get_data_frames()[1]

        name_list = []
        id_list = []
        for player in range(0, len(active_team_player_raw["VS_PLAYER_NAME"])):
            
           
            name_list.append(active_team_player_raw["VS_PLAYER_NAME"][player])
            id_list.append(active_team_player_raw["VS_PLAYER_ID"][player])
            
        
        ActivePlayer_List = {
            "Name": name_list,
            "Player ID": id_list
        }
       
        ActivePlayer_DataFrame = pd.DataFrame(data=ActivePlayer_List)
        
        return(ActivePlayer_DataFrame)
        

    def compile_active_team_player_data(self):
        
        team_roster_DataFrame = self.compile_team_roster_DataFrame()
        print("This is it")
        
        player_data = []
        for i in range(0, len(team_roster_DataFrame["Name"])):
            player_object = team_roster_DataFrame["Name"][i]
            print("initial", player_object)
            player_id= team_roster_DataFrame["Player ID"][i]
            player_name= team_roster_DataFrame["Name"][i]
            print("iteration", i)
            print(player_id, type(player_id))
            try:
                player_object = Player(player_id, self.TeamID) #TODO: Make Player ID auto
                print(player_object, "second time")
                print("In the try ")
                player_data.append(player_object.get_active_player_data())
            except:
                print("Error for", player_name)
            
        print(player_data)
        player_data = {
            "Player Data": player_data
        }
        player_data_DataFrame = pd.DataFrame(data=player_data)
        
        #print(player_data)
        return(player_data_DataFrame)
        #player_data_DataFrame["Name"] = player_DataFrame[]
            
   
    



In [8]:
test = Team("Portland Trail Blazers")

test.compile_active_team_player_data()



It
This is it
initial Anthony, Carmelo
iteration 0
2546 <class 'numpy.int64'>
6666666666666666666666666666666666666666666666666666666666
7777777777777777777777777777777777777777
1111111111111111111111111111111111111111
Carmelo Anthony
5555555555555555555555555555555555555555
gggggggggggggggggggggg 2003
88888888888888888888888
99999999999999999999999999999999999999999999999999
10101010101010101010101010100101010101010101
<__main__.Player object at 0x000001BEECF790F0> second time
In the try 
1111
Why is this not printing
222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
2010 2021
-------------------------------------------------
***************************************
5656656656566556565656556565656565556565656565656656556565655656566556
2010-11
767677676767676767676767676776
-------------------------------------------------
***************************************
5656656656566556565656556565656565556565656565656656556565655656566556
2011

AttributeError: 'list' object has no attribute 'tail'